In [ ]:
!pip install tqdm

In [ ]:
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm

## get_total_cost 함수

In [ ]:
distance_beta = np.array([[1,3,5,7,8,6,3],
                          [2,2,3,7,9,4,3]])
print(np.min(distance_beta, axis=1))

[1 2]


In [ ]:
def get_total_cost_with_Weight(distance_Matrix, demand_weight_Vector):
  distance_Matrix = np.min(distance_Matrix, axis=1)
  return distance_Matrix.dot(demand_weight_Vector)

In [ ]:
get_total_cost_with_Weight(distance_beta, np.array([1,2]))

5

## get_distance 함수

In [ ]:
!pip install haversine
from haversine import haversine

In [ ]:
# 위경도 입력
Seoul = [37.541, 126.986]  #Latitude, Longitude
Toronto = [43.65, -79.38]

# 거리 계산
haversine(Seoul, Toronto, unit = 'm')

10603855.192138562

In [ ]:
def get_distance(demand_location_List, N_station_location_List):
  distance_Matrix = []
  for demand in demand_location_List:
    distance_Matrix.append([haversine(demand, station, unit='m') for station in N_station_location_List])
  return distance_Matrix

In [ ]:
demand_location_list_Beta = [[37.541, 126.986], [37.540, 126.981], [35.541, 130.986]]
station_location_list_Beta = [[43.65, -79.38], [50, -79.38], [43.65, -63.38]]

In [ ]:
print(get_distance(demand_location_list_Beta, station_location_list_Beta))

[[10603855.192138562, 9943116.529719213, 10926746.528364778], [10604102.709269775, 9943348.939253071, 10926914.742524207], [10689054.90602304, 10045734.825619422, 11090258.973170951]]


## 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 반복문으로 후보 N개씩 함수에 대입하여 모든 Cost 계산

In [ ]:
pool = ['A', 'B', 'C']
print(list(itertools.permutations(pool))) # 3개의 원소로 수열 만들기
print(list(map(''.join, itertools.permutations(pool, 2)))) # 2개의 원소로 수열 만들기

In [ ]:
def get_cost_List_for_N_stations(demand_location_List, station_location_List, demand_weight_Vector, N):
  rst_List = []
  N_station_List = list(itertools.combinations(station_location_List, N))
  for i in tqdm(range(len(N_station_List))):
    distance_Matrix = get_distance(demand_location_List=demand_location_List, N_station_location_List=N_station_List[i])
    cost = get_total_cost_with_Weight(distance_Matrix=distance_Matrix, demand_weight_Vector=demand_weight_Vector)
    rst_List.append((N_station_List[i], cost))
    # print(f'getting cost... {}/{len(N_station_List)}')
  #   distance_Matrix = [get_distance(N_station, demand_location) for demand_location in demand_location_List]
  #   distance_Matrix = get_distance(demand_location_List, N_station)
  #   cost = get_total_Cost(distance_Matrix, demand_weight_Vector)
  #   rst_List.append([N_station, cost])
  return pd.DataFrame(rst_List)

## 관저동/도마동/봉명동/용전동 뽑기



In [ ]:
demand_List = pd.read_csv('/content/drive/MyDrive/공빅데/2.법정동_격자별_환승_및_정류장_갯수/용전동_최종.csv', encoding='cp949').fillna(0)
station_List = pd.read_csv('/content/drive/MyDrive/용전동_트램.csv', encoding='cp949').fillna(0)

In [ ]:
demand_List.head(3)

,left,bottom,right,top,NUMPOINTS,BUS_NN_max,탑승_sum,하차_sum,환승_sum,lon,lat
0,993400.0,1817500.0,993500.0,1817600.0,1.0,한숲아파트,20465.0,7331.0,443.0,127.426996,36.355345
1,993500.0,1816900.0,993600.0,1817000.0,0.0,0,0.0,0.0,0.0,127.428116,36.349936
2,993500.0,1817000.0,993600.0,1817100.0,0.0,0,0.0,0.0,0.0,127.428115,36.350838


In [ ]:
station_List.head(3)

,LINK_ID,F_NODE,T_NODE,LANES,ROAD_NAME,CONNECT,MAX_SPD,angle,lon,lat
0,1830040400,1830016200,1830015200,4,동대전로,0,60,149.872301,127.442019,36.351054
1,1830043800,1830016600,1830016200,4,한밭대로,0,60,134.713498,127.441407,36.351562
2,1830045100,1830016600,1830016900,4,한밭대로,0,60,315.981362,127.441502,36.351638


In [ ]:
demand_location_List = demand_List[['lon', 'lat']].to_numpy()

In [ ]:
station_location_List = station_List[['lon', 'lat']].to_numpy()

In [ ]:
weight_Vector = demand_List['환승_sum'].to_numpy()

In [ ]:
rst_1 = get_cost_List_for_N_stations(demand_location_List=demand_location_List, station_location_List= station_location_List, demand_weight_Vector= weight_Vector, N=1)
rst_1.to_csv('Yongjeon_P_Median_1.csv')
rst_2 = get_cost_List_for_N_stations(demand_location_List=demand_location_List, station_location_List= station_location_List, demand_weight_Vector= weight_Vector, N=2)
rst_2.to_csv('Yongjeon_P_Median_2.csv')
rst_3 = get_cost_List_for_N_stations(demand_location_List=demand_location_List, station_location_List= station_location_List, demand_weight_Vector= weight_Vector, N=3)
rst_3.to_csv('Yongjeon_P_Median_3.csv')

100%|██████████| 7770/7770 [00:21<00:00, 357.20it/s]
